In [1]:
from numpy import *
import time 
import numpy as np

In [3]:
x = random.random((18000,18000))
v = random.random((18000))

%timeit foo = dot(x, v)


10 loops, best of 3: 87.3 ms per loop


In [4]:
# test sparse matrix multiplication
from scipy import sparse

x1 = sparse.rand(18000,18000)
x1=x1.tobsr()

%timeit foo = x1.dot(v)


100 loops, best of 3: 3.84 ms per loop


In [5]:
def SpMV_viaMKL( A, x ):
     '''
     Wrapper to Intel's SpMV
     (Sparse Matrix-Vector multiply)
     For medium-sized matrices, this is 4x faster
     than scipy's default implementation
     Stephen Becker, April 24 2014
     stephen.beckr@gmail.com
     '''

     import numpy as np
     import scipy.sparse as sparse
     from ctypes import POINTER,c_void_p,c_int,c_char,c_double,byref,cdll
     mkl = cdll.LoadLibrary("libmkl_rt.so")

     SpMV = mkl.mkl_cspblas_dcsrgemv
     # Dissecting the "cspblas_dcsrgemv" name:
     # "c" - for "c-blas" like interface (as opposed to fortran)
     #    Also means expects sparse arrays to use 0-based indexing, which python does
     # "sp"  for sparse
     # "d"   for double-precision
     # "csr" for compressed row format
     # "ge"  for "general", e.g., the matrix has no special structure such as symmetry
     # "mv"  for "matrix-vector" multiply

     m, n = A.shape

     # The data of the matrix
     data    = A.data.ctypes.data_as(POINTER(c_double))
     indptr  = A.indptr.ctypes.data_as(POINTER(c_int))
     indices = A.indices.ctypes.data_as(POINTER(c_int))

     # Allocate output, using same conventions as input
     nVectors = 1
     if x.ndim is 1:
        y = np.empty(m,dtype=np.double,order='F')
     elif x.shape[1] is 1:
        y = np.empty((m,1),dtype=np.double,order='F')
        

   

     np_x = x.ctypes.data_as(POINTER(c_double))
     np_y = y.ctypes.data_as(POINTER(c_double))
     # now call MKL. This returns the answer in np_y, which links to y
     SpMV(byref(c_char("N")), byref(c_int(m)),data ,indptr, indices, np_x, np_y ) 

     return y

In [6]:
x2  = x1.tocsr()

%timeit foo = SpMV_viaMKL(A=x2,x=v)


1000 loops, best of 3: 1.59 ms per loop


In [7]:
import pyculib.sparse as pcu

x4 = sparse.rand(18000,18000)
x4= pcu.bsr_matrix(x4)



In [8]:
GPU = pcu.Sparse(0)

In [9]:
m,n = x4.shape
nnz = x4.nnz
descr = GPU.matdescr()
csrVal = x4.data
csrRowPtr = x4.indptr
csrColInd = x4.indices

y = np.empty(m,dtype=np.double)

In [10]:
%timeit GPU.csrmv('N', m, n, nnz, 1.0, descr, csrVal, csrRowPtr, csrColInd, v, 0.0, y)

100 loops, best of 3: 2.51 ms per loop


In [ ]:
y

In [11]:
GPU.stream

0

In [12]:
GPU1 = pcu.Sparse(0)

In [13]:
GPU1.stream

0

In [14]:
GPU.dense2csc